# Workshop

Introduction to Langchain

In [1]:
# Load OpenAI key from env

import os
from dotenv import load_dotenv
load_dotenv(override=True)

azure_version = "2024-06-01"
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_embeddings = os.getenv("AZURE_OPENAI_EMBEDDINGS")
azure_whisper = os.getenv("AZURE_OPENAI_WHISPER")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_key = os.getenv("AZURE_OPENAI_KEY")

In [2]:
# create llm instance

from langchain_openai.chat_models import AzureChatOpenAI

## unser llm ohne frontend
llm = AzureChatOpenAI(
    api_key=azure_key,
    api_version=azure_version,
    azure_deployment=azure_deployment,
    model=azure_deployment,
    azure_endpoint=azure_endpoint,
)

In [3]:
# invoke llm

llm.invoke("What do you know about Conferences?")

AIMessage(content="Conferences are structured events where individuals with shared interests gather to discuss topics, share insights, present research, and network. They can vary widely in scale, format, and subject matter, encompassing fields such as business, technology, science, education, and the arts. Here are some key points about conferences:\n\n1. **Types of Conferences**:\n   - **Academic Conferences**: Scholars and researchers present papers and discuss their findings. Examples include the American Association for the Advancement of Science (AAAS) Annual Meeting, and the Association for Computing Machinery (ACM) conferences.\n   - **Business Conferences**: Focus on industry trends, networking, and professional development. Examples include the World Economic Forum and TED conferences.\n   - **Technology Conferences**: Highlight innovations, product launches, and tech discussions. Examples include CES (Consumer Electronics Show) and Apple's WWDC (Worldwide Developers Conferen

In [4]:
# invoke llm

# hier kann ich das llm auch direkt fragen
answer = llm.invoke("What do you know about Conferences?")
print(answer.content)

Conferences are formal meetings or gatherings where individuals with shared interests, professions, or industries come together to discuss topics of mutual concern, share knowledge, network, and collaborate. They can vary widely in scope, size, and format, but generally, they serve several key purposes:

1. **Knowledge Sharing**: Conferences provide a platform for presenting new research, innovations, and developments. Speakers, often experts in their fields, deliver presentations, workshops, and panels to disseminate information.

2. **Networking**: Attendees have the opportunity to meet peers, mentors, potential collaborators, and industry leaders. Networking sessions, social events, and informal gatherings facilitate relationship-building.

3. **Professional Development**: Many conferences offer training sessions, workshops, and continuing education opportunities to help attendees enhance their skills and stay current in their fields.

4. **Industry Trends**: Conferences often highl

In [5]:
# use prompts

# hier frage ich ein Prompttemplate ab
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""You are an encyclopedia.

Question: {input}""")

chain = prompt | llm

answer = chain.invoke({"input": "What are Conferences?"})
print(answer.content)

Conferences are formal meetings or gatherings where individuals, often from a specific field or industry, come together to discuss particular topics, share research, exchange ideas, and network. They can vary widely in size, format, and purpose, and they often include presentations, panel discussions, workshops, and keynote speeches.

Types of conferences include:

1. **Academic Conferences**: These are typically organized by universities or academic organizations and focus on presenting and discussing research findings. Participants often include scholars, researchers, and students.

2. **Professional or Industry Conferences**: These are organized by professional associations or industry groups and focus on topics relevant to a specific industry or profession. They often feature industry leaders, product demonstrations, and continuing education opportunities.

3. **Business Conferences**: These are aimed at business professionals and may focus on topics such as management, leadership,

In [6]:
# reusue prompts

answer = chain.invoke({"input": "What are Conferences also called?"})
print(answer.content)



Conferences are also called conventions, symposiums, seminars, summits, forums, congresses, or meetings. Each term might have a slightly different connotation depending on the context and the specific nature of the gathering, but they all refer to assemblies of people who come together to discuss specific topics or issues.


In [7]:
# contexts

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

chain = prompt | llm

answer = chain.invoke(
    {
        "input": "What are Conferences also called?",
        "context": "An academic conference or scientific conference (also congress, symposium, workshop, or meeting) is an event for researchers (not necessarily academics) to present and discuss their scholarly work. Together with academic or scientific journals and preprint archives, conferences provide an important channel for exchange of information between researchers. Further benefits of participating in academic conferences include learning effects in terms of presentation skills and 'academic habitus', receiving feedback from peers for one's own research, the possibility to engage in informal communication with peers about work opportunities and collaborations, and getting an overview of current research in one or more disciplines.",
    }
)
print(answer.content)

Conferences are also called congresses, symposiums, workshops, or meetings.


In [8]:
# documents

from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain


documents = [
    Document(
        page_content="An academic conference or scientific conference (also congress, symposium, workshop, or meeting) is an event for researchers (not necessarily academics) to present and discuss their scholarly work. Together with academic or scientific journals and preprint archives, conferences provide an important channel for exchange of information between researchers. Further benefits of participating in academic conferences include learning effects in terms of presentation skills and 'academic habitus', receiving feedback from peers for one's own research, the possibility to engage in informal communication with peers about work opportunities and collaborations, and getting an overview of current research in one or more disciplines.",
        metadata={
            "source": "wikipedia"
        }
    ),
    Document(
        page_content="Die Data2Day ist eine Konferenz zum fachlichen Austausch zum Thema Data and AI.",
        metadata={
            "source": "Drenize"
        }
    )
]

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

answer = document_chain.invoke(
    {
        "input": "What are Conferences also called?",
        "context": documents,
    }
)
print(answer)

Conferences are also called congresses, symposiums, workshops, or meetings.


In [9]:
document_chain.invoke(
    {
        "input": "Is Data2Day a conference?", 
        "context": documents
    }
)

'Yes, Data2Day is a conference.'

In [10]:
answer = document_chain.invoke(
    {
        "input": "What are Conferences also called and what is the source?",
        "context": documents,
    }
)
print(answer)

Conferences are also called congresses, symposiums, workshops, or meetings. The source is the provided context.


In [11]:
# document prompts

document_prompt = ChatPromptTemplate.from_template("""Content: {page_content}                             
Source: {source}""")

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
    document_prompt=document_prompt,
)

answer = document_chain.invoke(
    {
        "input": "What are Conferences also called and what is the source?",
        "context": documents,
    }
)
print(answer)

Conferences are also called congresses, symposiums, workshops, or meetings. The source for this information is Wikipedia.


In [12]:
document_chain.invoke(
    {
        "input": "What is the meaning of conferences and what is the source?",
        "context": documents,
    }
)

'Conferences are events for researchers (not necessarily academics) to present and discuss their scholarly work. They provide an important channel for the exchange of information between researchers, offer benefits like learning presentation skills, receiving feedback, engaging in informal communication about work opportunities and collaborations, and getting an overview of current research in one or more disciplines. The source of this information is Wikipedia.'

## Vector Stores

In [42]:
##from langchain_community.document_loaders import PyPDFLoader
##
##file_path = (
##    "/Users/drenizerama/Blume Repo/rag-systems/Workshop-GenAI/Data/The Managers Path.pdf"
##)
##loader = PyPDFLoader(file_path)
##pages = loader.load_and_split()
##
##pages[0]

Document(metadata={'source': '/Users/drenizerama/Blume Repo/rag-systems/Workshop-GenAI/Data/The Managers Path.pdf', 'page': 1}, page_content='Praise\tfor\t\nThe\tManager’s\tPath\nThe\tManager’s\tPath\n\tgives\tthe\tbig\tpicture\tperspective\ton\twhat\ta\tcareer\tin\nengineering\tmanagement\tlooks\tlike.\tCamille\tprovides\tvery\ttactical\tadvice\nfor\teach\tcareer\tstage.\tAnd\tbecause\tengineering\tmanagers\thave\ta\tgreat\nresponsibility\tto\ttheir\treports\tto\tlearn\thow\tto\tmanage\twell,\tyou\tshould\tread\nthis\tbook\tand\tlearn\thow\tit\tis\tdone.\nThis\tbook\tis\ta\tpractical\tguide\tto\tunderstanding\tand\tpursuing\ta\tcareer\tin\nEngineering\tManagement\n.\nLiz\tCrawford,\tEntrepreneur\tin\tResidence,\tGenacast\tVentures;\t\nformer\tCTO,\nBirchbox\nAs\tCamille\tsays\tin\tChapter\t5,\t“This\tbook\tis\tfor\tengineering\tmanagers.\tIt’s\nnot\ta\tgeneric\tmanagement\tbook.”\tWithout\thesitation\tI\trecommend\tthis\nbook\tfor\tliterally\teveryone\twho\tworks\tin\tor\taround\tsoft

In [17]:
# vector stores

from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import AzureOpenAIEmbeddings

#vector datenbank
from langchain_chroma import Chroma


# load data
# statt wie oben die dokumente reinzuschreiben, laden wir die dokumente als elemente in eine liste
loader = TextLoader(r"./Data/Internal/budget.txt", encoding="utf-8")
data = loader.load()
#loader = TextLoader(r"./Data/Internal/lostpassword.txt", encoding="utf-8")
#data.extend(loader.load())
# PDF reinladen
loader = PyPDFLoader(r"/Users/drenizerama/Blume Repo/rag-systems/Workshop-GenAI/Data/thinkbayes.pdf")
data.extend(loader.load())

splitter = RecursiveCharacterTextSplitter(chunk_size=30000, chunk_overlap=1000, separators=[".", "\n"])
documents = splitter.split_documents(data)

# statt llm ist es hier embeddings
embeddings = AzureOpenAIEmbeddings(
    api_key=azure_key,
    api_version=azure_version,
    azure_deployment=azure_embeddings,
    azure_endpoint=azure_endpoint,
)
db = Chroma.from_documents(documents, embeddings, persist_directory="./chroma/presentation")

for document in documents:
    print(document)
    print("------------------------")

page_content='Tech Innovators Inc. Team Event Budget Application Guide

1. Introduction Organizing team events is a great way to boost morale and foster collaboration. This guide will help you navigate the process of applying for a budget for your team event.

2. Plan Your Event

Define the Purpose: Clearly outline the purpose of the event (e.g., team building, training, celebration).
Set Objectives: Identify the goals you aim to achieve with the event.
Draft a Proposal: Prepare a detailed proposal including the event’s purpose, objectives, and expected outcomes.
3. Estimate the Budget

List Expenses: Itemize all potential expenses, such as venue rental, catering, transportation, materials, and any other costs.
Get Quotes: Obtain quotes from vendors to ensure accurate budgeting.
Create a Budget Plan: Summarize the total estimated cost and break it down into categories.
4. Submit a Budget Request

Step 1: Access the HR Portal
Log in to the Tech Innovators HR portal using your employee c

In [19]:
### bayes beispiel

from langchain.chains.retrieval import create_retrieval_chain

# lokal abspeichern?
db = Chroma(persist_directory="./chroma/presentation", embedding_function=embeddings)
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [20]:

answer = retrieval_chain.invoke({"input": "What is bayesian thinking?"})
print(answer["answer"])

Bayesian thinking involves updating the probability of a hypothesis based on new data, which is encapsulated in Bayes's theorem. This approach starts with an initial probability (prior) and adjusts it in light of new evidence (likelihood) to form an updated probability (posterior). Bayesian thinking is a way to incorporate new information over time to refine the understanding or prediction of a hypothesis.


In [14]:
# retriever

from langchain.chains.retrieval import create_retrieval_chain

db = Chroma(persist_directory="./chroma/presentation", embedding_function=embeddings)
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

answer = retrieval_chain.invoke({"input": "What is the title of step two when losing a password?"})
print(answer["answer"])

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


The title of step two when losing a password is "Access the Password Recovery Page."


In [15]:
answer = retrieval_chain.invoke({"input": "Give a concise description how I can apply for a budget."})
print(answer["answer"])

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


To apply for a budget for a team event at Tech Innovators Inc., follow these steps:

1. **Plan Your Event**:
   - Define the purpose and set objectives.
   - Draft a detailed proposal with the event’s purpose, objectives, and expected outcomes.

2. **Estimate the Budget**:
   - List all potential expenses and obtain quotes from vendors.
   - Create a budget plan summarizing the total estimated cost.

3. **Submit a Budget Request**:
   - Log in to the Tech Innovators HR portal using your credentials.
   - Navigate to the “Budget Request” section and fill out the form with event details.
   - Attach your budget plan and supporting documents, then submit the request.

4. **Review and Approval**:
   - HR and finance departments will review your request for alignment with policies and reasonable budgeting.
   - You will receive an email notification of approval or denial.

5. **Coordinate with HR and Finance**:
   - Follow up with HR and coordinate with finance to finalize the budget and al

## Summary

In [16]:
summary_prompt = ChatPromptTemplate.from_template("""Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand.

Text: {context}""")

summary_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=summary_prompt,
    document_prompt=document_prompt,
)

answer = summary_chain.invoke(
    {"context": documents}
)
print(answer)

Hey kiddo! Let me tell you a story about two guides from a company called Tech Innovators.

First, there's a guide about planning a fun party for your team. Imagine you and your friends want to throw a big party, but you need some money to do it. This guide helps you ask the grown-ups (like your teachers) for money to buy things like decorations, snacks, and games. You need to write down why you want to have the party, what you hope to do, and how much everything will cost. Then, you ask the grown-ups for approval, and if they say yes, you can start planning and having fun!

Second, there's a guide about what to do if you forget your secret code to get into a special clubhouse. If you forget your code, you can click on a “Forgot Password” button, and they'll send you a special link to change it. You have to prove it's really you by answering some questions or entering a special code. Once you make a new secret code, you can get back into the clubhouse and play again. 

Both guides also

In [17]:
from functools import partial

from langchain.chains.combine_documents import collapse_docs, split_list_of_docs
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

partial_format_document = partial(format_document, prompt=document_prompt)

# The chain we'll apply to each individual document.
# Returns a summary of the document.
map_chain = (
    {"context": partial_format_document}
    | summary_prompt
    | llm
    | StrOutputParser()
)

# A wrapper chain to keep the original Document metadata
map_as_doc_chain = (
    RunnableParallel({"doc": RunnablePassthrough(), "content": map_chain})
    | (lambda x: Document(page_content=x["content"], metadata=x["doc"].metadata))
).with_config(run_name="Summarize (return doc)")

# The chain we'll repeatedly apply to collapse subsets of the documents
# into a consolidate document until the total token size of our
# documents is below some max size.
def format_docs(docs):
    return "\n\n".join(partial_format_document(doc) for doc in docs)

collapse_chain = (
    {"context": format_docs}
    | PromptTemplate.from_template("Collapse this content:\n\n{context}")
    | llm
    | StrOutputParser()
)

def get_num_tokens(docs):
    return llm.get_num_tokens(format_docs(docs))

def collapse(
    docs,
    config,
    token_max=400,
):
    collapse_ct = 1
    while get_num_tokens(docs) > token_max:
        config["run_name"] = f"Collapse {collapse_ct}"
        invoke = partial(collapse_chain.invoke, config=config)
        split_docs = split_list_of_docs(docs, get_num_tokens, token_max)
        docs = [collapse_docs(_docs, invoke) for _docs in split_docs]
        collapse_ct += 1
    return docs

# The chain we'll use to combine our individual document summaries
# (or summaries over subset of documents if we had to collapse the map results)
# into a final summary.

reduce_chain = (
    {"context": format_docs}
    | PromptTemplate.from_template("Combine these summaries:\n\n{context}")
    | llm
    | StrOutputParser()
).with_config(run_name="Reduce")

# The final full chain
map_reduce = (map_as_doc_chain.map() | collapse | reduce_chain).with_config(run_name="Map reduce")

answer = map_reduce.invoke(
    input=documents,
    config={"max_concurrency": 5},
)
print(answer)

**Combined Summary of Planning a Team Party and Handling a Forgotten Work Password:**

**Planning a Team Party:**

1. **Purpose**: Enhance team cohesion and enjoyment.
2. **Planning**:
   - Define the party's purpose.
   - List activities.
   - Develop a detailed plan.
3. **Budgeting**:
   - List purchases.
   - Determine costs.
   - Calculate total budget.
4. **Requesting Funds**:
   - Log into company site.
   - Navigate to funding request.
   - Submit form with details and costs.
5. **Approval**:
   - HR reviews concept.
   - Finance evaluates budget.
   - Await funding confirmation.
6. **Coordination**:
   - Contact HR for queries.
   - Collaborate with finance on budget.
7. **Preparation**:
   - Confirm vendor details.
   - Inform team of logistics.
8. **Post-Party**:
   - Submit receipts for reimbursement.
   - Write event report.
9. **Assistance**:
   - Contact HR for help.

**Handling a Forgotten Work Password:**

1. **Visit the Login Page**.
2. **Select “Forgot Password”**.
3.

## Few shotting

In [18]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.
Now tell me which intent the following question has.
                                            
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

The intent of the question "Let's go have some food at Wendy's." is ordering food.


In [19]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.
Now tell me which intent the following question has. Only answer with two/three words.
                                            
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

Ordering food


In [20]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.

Consider the following examples:

Question: Let's go to Tokyo next week.
Answer: Booking a flight

Question: Get some pizza delivered for this evening.
Answer: Ordering food

Question: How about a movie at 8pm at the cinema?
Anmswer: Getting a reservation

Now tell me which intent the follolwing question has. Only answer with two/three words.
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

Getting a reservation


## Agent and Tools

In [21]:
answer = llm.invoke("What did Frank-Walter Steinmeier say in his christmas speech 2023?")
print(answer.content)

As of now, the content of Frank-Walter Steinmeier's Christmas speech for 2023 is not available. If you need information on this topic, I recommend checking the latest news sources or the official website of the German Federal President closer to or after Christmas 2023 for the most accurate and up-to-date information.


In [29]:
from langchain.tools import Tool
from langchain.tools.ddg_search import DuckDuckGoSearchRun
#from langchain.tools.google_scholar import GoogleScholarQueryRun

ddg = DuckDuckGoSearchRun()

ddg_tool = Tool.from_function(
    func = ddg.run,
    name = "DuckDuckGo Search",
    description = "Search DuckDuckGo for a query about current events.",
)

#gsq= GoogleScholarQueryRun()

## en
#gsq_tool = Tool.from_function(
#    func = gsq.run,
#    name = "GoogleScholarQueryRun Search",
#    description = "Search GoogleScholarQueryRun for a query about current events.",
#)

#Liste von Tools
tools = [ddg_tool] #wird erst später verwendet vom Agent

ddg_tool.run("What did Frank-Walter Steinmeier say in his christmas speech 2023?")

'In his 2023 Christmas address, Federal President Steinmeier appealed to stand and stay together: "Let us trust in the strength and experience within us", he said in the message broadcasted on 25 December. "As rational, responsible individuals, we can together cope with a world that challenges us." 12/25/2023. In his Christmas speech, German President Frank-Walter Steinmeier focused on the need for courage and cooperation in the face of challenges in Germany and around the world. Advertisement. Read the speech online: www.bundespräsident.de Page 1 of 3 Christmas address by Federal President Frank-Walter Steinmeier Schloss Bellevue, 25 December 2023 "I\'ve stopped watching the news." That sentence is probably the one I have heard most often this year. Again and again, wherever I have been, people have expressed 12/25/2023 December 25, 2023. In his annual Christmas television address to the nation, German President Frank-Walter Steinmeier touched on the many conflicts and crises that dom

In [24]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate

agent_prompt = PromptTemplate.from_template("""
You are a great AI-Assistant. You have access to tools to help you answer questions:

{tools}

To use a tool, please use the following format:

'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 3 times)
'''

When you have a response to say or question to ask to the Human, or if you do not need to use a tool, you MUST use the format:
'''
Final Thought: Do I need to use a tool? No
Final Answer: [your response or question here]
'''

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")
agent = create_react_agent(
    tools=tools, llm=llm, prompt=agent_prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, max_iterations=30, verbose=True)

agent_executor.invoke({"input": "What did Frank-Walter Steinmeier say in his christmas speech 2023?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


'''
Thought: Do I need to use a tool? Yes
Action: DuckDuckGo Search
Action Input: Frank-Walter Steinmeier Christmas speech 2023
'''In his 2023 Christmas address, Federal President Steinmeier appealed to stand and stay together: "Let us trust in the strength and experience within us", he said in the message broadcasted on 25 December. "As rational, responsible individuals, we can together cope with a world that challenges us." Read the speech online: www.bundespräsident.de Page 1 of 3 Christmas address by Federal President Frank-Walter Steinmeier Schloss Bellevue, 25 December 2023 "I've stopped watching the news." That sentence is probably the one I have heard most often this year. Again and again, wherever I have been, people have expressed 12/25/2023. In his Christmas speech, German President Frank-Walter Steinmeier focused on the need for courage and cooperation in the face of challenges in Germany and around the world. Advertisement. In seiner Weihnachtsansprache 2023 hat Bundespräs

{'input': 'What did Frank-Walter Steinmeier say in his christmas speech 2023?',
 'output': 'In his 2023 Christmas speech, German President Frank-Walter Steinmeier emphasized the importance of courage and cooperation in addressing challenges both within Germany and globally. He encouraged people to trust in their own strength and experience, and to work together as rational and responsible individuals to navigate a challenging world.'}

In [25]:
new_agent_prompt = PromptTemplate.from_template("""
You are a great AI-Assistant and only answer in 10 words. You have access to tools to help you answer questions:

{tools}

To use a tool, please use the following format:

'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 3 times)
'''

When you have a response to say or question to ask to the Human, or if you do not need to use a tool, you MUST use the format:
'''
Final Thought: Do I need to use a tool? No
Final Answer: [your response or question here]
'''

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")
agent = create_react_agent(
    tools=tools, llm=llm, prompt=new_agent_prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, max_iterations=30, verbose=True)


agent_executor.invoke({"input": "What did Frank-Walter Steinmeier say in his christmas speech 2023?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


'''
Thought: Do I need to use a tool? Yes
Action: DuckDuckGo Search
Action Input: Frank-Walter Steinmeier Christmas speech 2023In his 2023 Christmas address, Federal President Steinmeier appealed to stand and stay together: "Let us trust in the strength and experience within us", he said in the message broadcasted on 25 December. "As rational, responsible individuals, we can together cope with a world that challenges us." Read the speech online: www.bundespräsident.de Page 1 of 3 Christmas address by Federal President Frank-Walter Steinmeier Schloss Bellevue, 25 December 2023 "I've stopped watching the news." That sentence is probably the one I have heard most often this year. Again and again, wherever I have been, people have expressed 12/25/2023. In his Christmas speech, German President Frank-Walter Steinmeier focused on the need for courage and cooperation in the face of challenges in Germany and around the world. Advertisement. 12/25/2023 December 25, 2023. In his annual Christmas

{'input': 'What did Frank-Walter Steinmeier say in his christmas speech 2023?',
 'output': 'Steinmeier emphasized unity, courage, and cooperation in his 2023 speech.'}

In [26]:
from langchain_core.runnables import RunnablePassthrough

simple_summary_prompt = ChatPromptTemplate.from_template("""Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand. Keep it funky!

Text: {input}""")
simple_summary_chain = {"input": RunnablePassthrough()} | simple_summary_prompt | llm

summary_tool = Tool(
    name="Funky Summary Tool",
    func=simple_summary_chain.invoke,
    description="Use this tool to do a funky summary. Make sure you get the text to do a summary of first."
)
tools.append(summary_tool)

agent = create_react_agent(
    tools=tools, llm=llm, prompt=agent_prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, max_iterations=30, verbose=True)

agent_executor.invoke({"input": "Funky summarize the christmas speech 2023 of Frank-Walter Steinmeier?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: Do I need to use a tool? Yes
Action: DuckDuckGo Search
Action Input: "Frank-Walter Steinmeier Christmas speech 2023 transcript"In his 2023 Christmas address, Federal President Steinmeier appealed to stand and stay together: "Let us trust in the strength and experience within us", he said in the message broadcasted on 25 December. "As rational, responsible individuals, we can together cope with a world that challenges us." Read the speech online: www.bundespräsident.de Page 1 of 3 Christmas address by Federal President Frank-Walter Steinmeier Schloss Bellevue, 25 December 2023 "I've stopped watching the news." That sentence is probably the one I have heard most often this year. Again and again, wherever I have been, people have expressed In seiner Weihnachtsansprache 2023 hat Bundespräsident Steinmeier an das Miteinander appelliert: "Vertrauen wir der Stärke und der Erfahrung, die in uns steckt", sagte er in der Ansprache, die am 25. Dezember gesendet wurde. "Als vernünftige un

{'input': 'Funky summarize the christmas speech 2023 of Frank-Walter Steinmeier?',
 'output': 'Alright kiddo, here\'s the scoop! Imagine you and your friends are on a big adventure and things are getting a bit tricky. The big boss, President Frank-Walter Steinmeier, gave a super-duper Christmas pep talk. He said, "Hey team, let\'s stick together and be brave!"\n\nHe knows there\'s a lot of yucky stuff happening around, but he wants everyone to stop worrying so much. Instead, he wants us to remember that we have lots of smarts and strength inside us. If we work together, we can handle anything!\n\nSo, his message was all about teamwork, being responsible, and staying strong together, even when things get tough. High five to that! 🎄✨👊'}

In [27]:
answer = llm.invoke("What is 1000 + 1234?")
print(answer.content)

1000 + 1234 equals 2234.


In [28]:
answer = llm.invoke("What is 13 raised to the .3432 power?")
print(answer.content)

To calculate \( 13^{0.3432} \), you can use a calculator with an exponentiation function or a scientific calculator. Here's the calculation:

\[ 13^{0.3432} \approx 2.450 \]

So, \( 13 \) raised to the power of \( 0.3432 \) is approximately \( 2.450 \).


## Images

In [31]:
import base64
from mimetypes import guess_type
from langchain.prompts import HumanMessagePromptTemplate

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    mime_type, _ = guess_type(image_path)
    # Default to png
    if mime_type is None:
        mime_type = 'image/png'

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"


prompt_template =  HumanMessagePromptTemplate.from_template(
            template=[
                {"type": "text", "text": "Summarize this image"},
                {
                    "type": "image_url",
                    "image_url": "{encoded_image_url}",
                },
            ]
        )

summarize_image_prompt = ChatPromptTemplate.from_messages([prompt_template])

gpt4_image_chain = summarize_image_prompt | llm 

img_file = "Images/UserGuide.jpg"
page3_encoded = local_image_to_data_url(img_file)

answer = gpt4_image_chain.invoke(input={"encoded_image_url":page3_encoded})
print(answer.content)

The image depicts a futuristic smart home system setup by "Tech Invoaters." It showcases a high-tech living room where a person is seated on a couch, interacting with various smart devices. The room is filled with interconnected gadgets, screens, and control panels, illustrating a comprehensive smart home ecosystem. There are large digital displays on the walls showing home automation interfaces, security systems, and other smart home functionalities. The environment is sleek and modern, emphasizing the integration of technology into everyday home life. The image also includes a list of tools and components associated with the smart home system.


## Audio

In [30]:
import openai

client = openai.AzureOpenAI(
    api_key=azure_key,
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_whisper,
    api_version=azure_version,
)

def get_transcript(audio_file):
    if not os.path.exists(audio_file):
        audio_file = "./Audio/" + audio_file
    client.audio.with_raw_response
    return client.audio.transcriptions.create(
        file=open(audio_file, "rb"),            
        model="whisper",
        language="de",
    ).text

audio_test_file = "./Audio/newyear2023.mp3"
print(get_transcript(audio_test_file))


Liebe Mitbürgerinnen und liebe Mitbürger, wenn ich im Land unterwegs bin, sagen mir gerade auch viele Ältere, so geballt, so Schlag auf Schlag habe ich das alles noch niemals erlebt. Kaum war Corona halbwegs vorbei, brach Russland mitten in Europa einen unerbittlichen Krieg vom Zaun. Kurz darauf dreht uns der russische Präsident den Gasthahn ab. Und im Herbst gab es auch noch den brutalen Terrorangriff der Hamas auf Israel. So viel Leid, so viel Blut vergießen. Unsere Welt ist unruhiger und rauer geworden. Sie verändert sich in geradezu atemberaubender Geschwindigkeit. Auch wir müssen uns deshalb verändern. Vielen von uns bereitet das Sorgen. Bei einigen sorgt das auch für Unzufriedenheit. Ich nehme mir das zu Herzen. Und zugleich weiß ich, wir in Deutschland kommen da durch. Erinnern Sie sich, wo wir vor einem Jahr standen. Drei, vier, fünf Prozent Wirtschaftseinbruch hatten uns viele Expertinnen und Experten vorausgesagt. Viele befürchteten, die Preise würden immer weiter steigen. Es

In [32]:
# audio
def get_audio_doc(input):
    return {
        "context": [Document(
            page_content=get_transcript(input),
            metadata={
                "source": "audio"
            }
        )]
    }
audio_summary_chain = get_audio_doc | summary_chain
answer = audio_summary_chain.invoke("./Audio/newyear2023.mp3")
print(answer)


Hey kiddo, so this is like a big speech where a grown-up tells everyone in their country about the tough times they’ve been through. They talk about how there were some really hard things like a big sickness, a war, and some scary attacks. But even though things were tough, they all worked together to save energy and keep working. Because of that, things got a little better.

They also say that lots of people have jobs now, and that helps everyone have a good life. They’re planning to fix roads and trains, make cleaner energy, and create good jobs. They want to make sure everyone, even those who don’t earn a lot of money, get some help too.

The grown-up also talks about how important it is to have friends in other countries and to stick together, especially in Europe. They remind everyone how important it is to talk and work together to make good decisions. They thank all the helpers like firefighters and police who do a lot to keep everyone safe.

In the end, they say that everyone i